Integrating Chat Prompt Templates and Sessions

Prompt templates structure raw user input into a format suitable for the LLM, allowing for system instructions and specific message placeholders.

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from dotenv import load_dotenv
import os

In [19]:
prompt=ChatPromptTemplate.from_messages([
    ("ai","You are a helpful assistant to guide user with their queries in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

In [20]:
load_dotenv()

GROQ_API_KEY=os.getenv("GROQ_API_KEY")
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=GROQ_API_KEY)

In [21]:
chain=prompt|model

In [24]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(
   chain,
    get_session_history,
    input_messages_key="messages"
)

In [26]:
config1={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="Hello how are you")],
        "language":"Turkish"
    }
    ,config=config1
)
response

AIMessage(content="Merhaba, iyiyim, sen nasılsın? (Hello, I'm fine, how are you?)", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 57, 'total_tokens': 82, 'completion_time': 0.088332987, 'completion_tokens_details': None, 'prompt_time': 0.002962855, 'prompt_tokens_details': None, 'queue_time': 0.064938645, 'total_time': 0.091295842}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb310-e170-7fc2-9507-b87155fa4731-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 25, 'total_tokens': 82})